# Evaluating LLM performance in the presence of missing outputs

### *Research goal: use agentic LLMs to calculate RAI frailty scores with only clinical notes extracted from electronic health records*

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [ ]:
# Libraries and imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve, auc
import statsmodels.api as sm

all_df = pd.read_csv(r'C:\Users\yens01\Projects\Frailty\frailty_allraiscores.csv', index_col=0)
if 'age' not in all_df.columns:
    print("Age column not found, generating random ages for demonstration.")
    all_df['age'] = np.random.randint(18, 100, size = len(all_df))
all_df.keys()

In [ ]:
def get_specific_model_df(df, model_name):
    subset_df = df[['age', 'token_count', 'Validated_RAI_Score', model_name]].copy()
    subset_df.rename(columns={model_name: 'llm_RAI_score'}, inplace=True)
    return subset_df

# extract only columns relevant to llama3.1 single for this demonstration
scores_df = get_specific_model_df(all_df, 'llama3_1_single_RAI')
scores_df.head()

In [ ]:
n          = len(scores_df)
n_missing  = sum(scores_df['llm_RAI_score'].isna())
n_complete = n - n_missing

print(f"Number of complete cases: {n_complete}, Number of missing cases: {n_missing}, Total: {n}")

In [ ]:
mask_missing  = scores_df["llm_RAI_score"].isna()
mask_complete = scores_df["llm_RAI_score"].notna()

validated_missing  = scores_df.loc[mask_missing,  "Validated_RAI_Score"]
validated_complete = scores_df.loc[mask_complete, "Validated_RAI_Score"]
llm_missing        = scores_df.loc[mask_missing,  "llm_RAI_score"] # just NaNs lol
llm_complete       = scores_df.loc[mask_complete, "llm_RAI_score"]

scores_df['validated_frailty'] = (scores_df['Validated_RAI_Score'] >= 21)
scores_df['llm_frailty']       = (scores_df['llm_RAI_score'] >= 21)
validated_complete_frailty     = scores_df.loc[mask_complete, 'validated_frailty']
llm_complete_frailty           = scores_df.loc[mask_complete, 'llm_frailty']

In [ ]:
def calc_binary_metrics(y_true, y_pred):
    accuracy  = accuracy_score(y_true = y_true, y_pred = y_pred)
    precision = precision_score(y_true = y_true, y_pred = y_pred)
    recall    = recall_score(y_true = y_true, y_pred = y_pred)
    fone      = f1_score(y_true = y_true, y_pred = y_pred)
    prcurve  = precision_recall_curve(y_true = y_true, probas_pred = y_pred,
                                      pos_label = True)
    precisioncurve, recallcurve, _ = prcurve
    prauc = auc(x = recallcurve, y = precisioncurve)
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': fone,
        'pr_auc': prauc
    }

binary_metrics_complete = calc_binary_metrics(
    y_true = validated_complete_frailty,
    y_pred = llm_complete_frailty
)
binary_metrics_complete = {k: round(v, 4) for k, v in binary_metrics_complete.items()}
print("Binary metrics for complete cases:")
print(binary_metrics_complete)

In [ ]:
def calc_continuous_metrics(y_true, y_pred):
    spearman_corr, _ = stats.spearmanr(y_true, y_pred)
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))

    bootstrap_spear = bootstrap(
        data=(y_true, y_pred),
        statistic=lambda y_t, y_p: stats.spearmanr(y_t, y_p)[0],
        n_resamples=1000,
        random_state=1,
        method='basic'
    )
    ci_lower_spear, ci_upper_spear = bootstrap_spear.confidence_interval

    bootstrap_rmse = bootstrap(
        data=(y_true, y_pred),
        statistic=lambda y_t, y_p: np.sqrt(np.mean((y_t - y_p) ** 2)),
        n_resamples=1000,
        random_state=1,
        method='basic'
    )
    ci_lower_rmse, ci_upper_rmse = bootstrap_rmse.confidence_interval

    return {
        'spearman_corr': spearman_corr,
        'rmse': rmse,
        'ci_lower_spear': ci_lower_spear,
        'ci_upper_spear': ci_upper_spear,
        'ci_lower_rmse': ci_lower_rmse,
        'ci_upper_rmse': ci_upper_rmse
    }

continuous_metrics_complete = calc_continuous_metrics(
    y_true = validated_complete,
    y_pred = llm_complete
)
continuous_metrics_complete = {k: round(v, 4) for k, v in continuous_metrics_complete.items()}
print("Continuous metrics for complete cases:")
print(continuous_metrics_complete)

##### Context: to calculate RAI scores, zero-shot prompts were given to several different types of LLMs (no training done beyond the model's own pretraining). Even after several rounds of prompt engineering, some LLMs still occasionally failed to produce a valid output.

**Question:** what is the best way to report model performance metrics in this context?

**Proposal:** treat it like a classic missing data problem

### Step 1: Characterize Missingness

First, we need to decide if we can reasonably assume the data is:

* MCAR: LLM failure probability has nothing do with any observed or unobserved patient features
* MAR: LLM failure depends on patient feature(s), but not on patient's true frailty level
* MNAR: LLM failure depends on the true frailty level

In [ ]:
plt.figure(figsize = (5, 4))
plt.boxplot([validated_missing, validated_complete], 
            labels = ['Missing LLM Output', 'Complete LLM Output'])
plt.title('Validated RAI Score by LLM Output Missingness')
plt.ylabel('Validated RAI Score')
plt.show()

The missingness pattern is unclear visually. Let's run a significance test to determine if there's a material difference between the two groups.

First, let's do a visual check of the distributions of the two groups to see if we can use a t-test for the comparison.

In [ ]:
fig, axes = plt.subplots(nrows = 1, ncols = 2,
                         figsize = (10, 4), sharey = True)

bins = np.linspace(scores_df['Validated_RAI_Score'].min(),
                   scores_df['Validated_RAI_Score'].max(), 11)

axes[0].hist(validated_complete, bins = bins, density = True)
axes[0].set_title("Validated RAI Score (Complete LLM Output)")
axes[0].set_xlabel("RAI Score")
axes[0].set_ylabel("Probability Density")

axes[1].hist(validated_missing, bins = bins, density = True)
axes[1].set_title("Validated RAI Score (Missing LLM Output)")
axes[1].set_xlabel("RAI Score")

plt.show()

Seems like not too skewed, but just in case let's run both a permutation test and a t-test.

In [ ]:
# Permutation Test for Difference in Means
# Significance level: 0.05
# Null: missing and complete RAI scores are from the same distribution
# Alternative: different distributions
def mean_difference(x, y, axis):
    return np.mean(x, axis = axis) - np.mean(y, axis = axis)
permtest = stats.permutation_test(data = (validated_complete,
                                          validated_missing),
                                    statistic = mean_difference,
                                    vectorized = True)
print(f'Permutation test p-value: {round(permtest.pvalue, 4)}')

In [ ]:
# Unequal Variances T-test for Difference in Means
# Significance level: 0.05
# Null: missing and complete RAI scores are from the same distribution
# Alternative: different distributions
ttest = stats.ttest_ind(validated_complete, validated_missing, equal_var = False)
print(f'T-test p-value: {round(ttest.pvalue, 4)}')

Both of our tests show that at a 5% significance level, there's a difference between the two patient groups' RAI scores for complete vs missing LLM outputs. This means that the missingness is related to the outcome.

This is a violation of the assumptions of MCAR and MAR, which are required to many statistical missing data methodologies. This means that we cannot use the complete cases for analysis without biasing our results. We should consider using imputation methods or sensitivity analysis to account for the missing data.

In [ ]:
# logistic regression between missingness and token_count
X = sm.add_constant(scores_df['token_count'])
y = scores_df['llm_RAI_score'].isna().astype(int)
logit_model = sm.Logit(y, X)
logit_result = logit_model.fit(disp = False)
print("Logistic Regression Results:")
print(logit_result.summary())

Looks like there is a significant positive relationship between (log of) token count and missingness likelihood.

### Step 2: Identify Predictors of Missingness

The first step when we have MNAR data is to attempt to get higher quality data. :) We did this with some prompt engineering, but still have some missing LLM outputs.

To handle these, we first look at the associations between our true frailty scores and the patient features. The features we have access to are:

* LLM-predicted RAI score (where our missingness occurs)
* token count of clinical notes
* patient age

for each patient. As such, we'll investigate the following associations:

1. LLM-predicted RAI score vs token count
1. LLM-predicted RAI score vs age

If any of these associations prove material, then we would impute the data based on the relationship between LLM-predicted RAI score and that feature.

In [ ]:
# Plot LLM-predicted RAI score against token count
plt.figure(figsize = (5, 4))
plt.scatter(scores_df['token_count'], scores_df['llm_RAI_score'], alpha = 0.5)
plt.title('LLM-predicted RAI Score vs Token Count')
plt.xlabel('Token Count')
plt.ylabel('LLM-predicted RAI Score')
plt.xscale('log')
plt.show()

In [ ]:
# Regression of LLM-predicted RAI score on token count
olsmodel_token_count = sm.OLS(scores_df['llm_RAI_score'],
                              sm.add_constant(scores_df[['token_count']]),
                              missing = 'drop')
olsfit_token_count   = olsmodel_token_count.fit()
print(olsfit_token_count.summary())

In [ ]:
# Plot LLM-predicted RAI score against age
plt.figure(figsize = (5, 4))
plt.scatter(scores_df['age'], scores_df['llm_RAI_score'], alpha = 0.5)
plt.title('LLM-predicted RAI Score vs Age')
plt.xlabel('Age')
plt.ylabel('LLM-predicted RAI Score')
plt.show()

In [ ]:
# Linear regression of LLM-predicted RAI score on age
olsmodel_age = sm.OLS(scores_df['llm_RAI_score'],
                      sm.add_constant(scores_df['age']),
                      missing = 'drop')
olsfit_age   = olsmodel_age.fit()
print(olsfit_age.summary())

### Step 3: Imputation/Scenario Analysis

##### Option 1: Deterministic Regression Imputation

If either age or token count is a good linear predictor of llm_RAI_score, we can use that relationship to impute the missing values.

In [ ]:
print(f"P-values for token count regression coefficients: {round(olsfit_token_count.pvalues[1], 4)}")
print(f"P-values for age regression coefficients: {round(olsfit_age.pvalues[1], 4)}")
# token_count pval is below 0.05 significance level
# age pval is not (? TBD based on the actual ages) below 0.05 significance level

Token count looks to be a significant predictor of the LLM-predicted RAI score. We can thus use the regression results to predict llm_RAI_score for missing values.

In [ ]:
X_missing = sm.add_constant(scores_df.loc[mask_missing, 'token_count'])
imputed_token_counts = olsfit_token_count.predict(X_missing)

imputed_scores_df = scores_df.copy()
imputed_scores_df.loc[mask_missing, 'llm_RAI_score'] = imputed_token_counts

imputed_scores_df['validated_frailty'] = (imputed_scores_df['Validated_RAI_Score'] >= 21)
imputed_scores_df['llm_frailty']       = (imputed_scores_df['llm_RAI_score'] >= 21)

print(imputed_scores_df.head())

Then, we just rerun all our metrics on this imputed dataset.

In [ ]:
# Calculate binary classification metrics using function from before
binary_metrics_linear_imputation = calc_binary_metrics(
    y_true = imputed_scores_df['validated_frailty'],
    y_pred = imputed_scores_df['llm_frailty']
)
binary_metrics_linear_imputation = {k: round(v, 4) for k, v in binary_metrics_linear_imputation.items()}
print("Binary metrics for linearly imputed LLM RAI scores:")
print(binary_metrics_linear_imputation)

print("Binary metrics for complete cases:")
print(binary_metrics_complete)

In [ ]:
# Calculate continuous metrics using function from before
continuous_metrics_linear_imputation = calc_continuous_metrics(
    y_true = imputed_scores_df['Validated_RAI_Score'],
    y_pred = imputed_scores_df['llm_RAI_score']
)
continuous_metrics_linear_imputation = {k: round(v, 4) for k, v in continuous_metrics_linear_imputation.items()}
print("Continuous metrics for linearly imputed LLM RAI scores:")
print(continuous_metrics_linear_imputation)

print("Continuous metrics for complete cases:")
print(continuous_metrics_complete)

If age ends up being a factor as well, we can also run multiple linear regression and impute based on both age and token_count.

##### Option 2: Multiple Imputation

If neither age or token count are good predictors of llm_RAI_score (or if we want to get a sort of best/worst case scenario range for our metrics), we then impute the data multiple times with random plausible values to get a range on our numbers. The bounds/imputation mechanisms are informed by our knowledge about RAI scores.

We know:

- min and max possible RAI scores (by definition between 0 and 81)
- min and max RAI scores of our actual patients (0 and 47)

Since we're not 100% about which of these bounds is more appropriate, let's perform a sensitivity analysis, i.e. try both sets to assess the impact on our results.

In [ ]:
n_sims = 100
min_RAI, max_RAI = 0, 81

In [ ]:
min_validated_RAI = min(scores_df['Validated_RAI_Score'])
max_validated_RAI = max(scores_df['Validated_RAI_Score'])
print(min_validated_RAI, max_validated_RAI)

In [ ]:
def mult_impute_with_randint(df, mask_missing, min_val, max_val, n_sims = 100):
    imputed_dfs = []
    for i in range(n_sims):
        np.random.seed(i)
        imputed_df = df.copy()
        n_missing = mask_missing.sum()
        imputed_vals = np.random.randint(min_val, max_val + 1, size = n_missing)
        imputed_df.loc[mask_missing, 'llm_RAI_score'] = imputed_vals
        imputed_df['llm_frailty'] = (imputed_df['llm_RAI_score'] >= 21)
        imputed_df['validated_frailty'] = (imputed_df['Validated_RAI_Score'] >= 21)
        imputed_dfs.append(imputed_df)
    return imputed_dfs

In [ ]:
# Version 1: theoretical min/max
imputed_possible_all = mult_impute_with_randint(scores_df, mask_missing,
                                                min_RAI, max_RAI, n_sims)
# Version 2: min/max from validated RAI scores
imputed_validated_all = mult_impute_with_randint(scores_df, mask_missing,
                                                 min_validated_RAI,
                                                 max_validated_RAI, n_sims)

In [ ]:
def get_imputed_metrics_samps(imputed_dfs):
    spearman_samps, rmse_samps = [], []
    accuracy_samps, precision_samps, recall_samps = [], [], []
    f1_samps, pr_auc_samps = [], []

    for imputed_df in imputed_dfs:
        cont_met = calc_continuous_metrics(
            y_true = imputed_df['Validated_RAI_Score'],
            y_pred = imputed_df['llm_RAI_score']
        )
        bin_met = calc_binary_metrics(
            y_true = imputed_df['validated_frailty'],
            y_pred = imputed_df['llm_frailty']
        )
        spearman_samps.append(cont_met['spearman_corr'])
        rmse_samps.append(cont_met['rmse'])
        accuracy_samps.append(bin_met['accuracy'])
        precision_samps.append(bin_met['precision'])
        recall_samps.append(bin_met['recall'])
        f1_samps.append(bin_met['f1_score'])
        pr_auc_samps.append(bin_met['pr_auc'])
    return spearman_samps, rmse_samps, accuracy_samps, precision_samps, recall_samps, f1_samps, pr_auc_samps

In [ ]:
# For demo model:
spearman_samps_possible, rmse_samps_possible, accuracy_samps_possible, precision_samps_possible, recall_samps_possible, f1_samps_possible, pr_auc_samps_possible = get_imputed_metrics_samps(imputed_possible_all)
spearman_samps_validated, rmse_samps_validated, accuracy_samps_validated, precision_samps_validated, recall_samps_validated, f1_samps_validated, pr_auc_samps_validated = get_imputed_metrics_samps(imputed_validated_all)

In [ ]:
# For demo model:
print("Theoretical min/max imputation metrics:")
print(f"Accuracy: {round(np.mean(accuracy_samps_possible), 4)}")
print(f"Precision: {round(np.mean(precision_samps_possible), 4)}")
print(f"Recall: {round(np.mean(recall_samps_possible), 4)}")
print(f"F1 Score: {round(np.mean(f1_samps_possible), 4)}")
print(f"PR AUC: {round(np.mean(pr_auc_samps_possible), 4)}")
print(f"Spearman correlation: {round(np.mean(spearman_samps_possible), 4)}")
print(f"RMSE: {round(np.mean(rmse_samps_possible), 4)}")

print("\nValidated RAI min/max imputation metrics:")
print(f"Accuracy: {round(np.mean(accuracy_samps_validated), 4)}")
print(f"Precision: {round(np.mean(precision_samps_validated), 4)}")
print(f"Recall: {round(np.mean(recall_samps_validated), 4)}")
print(f"F1 Score: {round(np.mean(f1_samps_validated), 4)}")
print(f"PR AUC: {round(np.mean(pr_auc_samps_validated), 4)}")
print(f"Spearman correlation: {round(np.mean(spearman_samps_validated), 4)}")
print(f"RMSE: {round(np.mean(rmse_samps_validated), 4)}")

In [ ]:
# For all models:
all_model_names = [col for col in all_df.columns if col.endswith('agent_RAI') or col.endswith('single_RAI')]
all_model_names.sort()
print(all_model_names)

In [ ]:
def round_metrics(metrics):
    return {k: round(v, 4) for k, v in metrics.items()}

In [ ]:
def print_metrics_for_all_models(all_df, all_model_names, min_val, max_val, n_sims = 100):
    for model_name in all_model_names:
        scores_df = get_specific_model_df(all_df, model_name)
        n = len(scores_df)
        n_missing = sum(scores_df['llm_RAI_score'].isna())
        n_complete = n - n_missing
        print(f"Model: {model_name}")
        print(f"Number of complete cases: {n_complete}, Number of missing cases: {n_missing}, Total: {n}")
        print(f"Failure percentage for {model_name}: {(n_missing / n) * 100:.2f}%")

        if n_missing == 0:
            print(f"No missing cases for {model_name}. Skipping imputation.")
            scores_df['validated_frailty'] = (scores_df['Validated_RAI_Score'] >= 21)
            scores_df['llm_frailty']       = (scores_df['llm_RAI_score'] >= 21)
            binary_metrics_complete = calc_binary_metrics(
                y_true = scores_df['validated_frailty'],
                y_pred = scores_df['llm_frailty']
            )
            continuous_metrics_complete = calc_continuous_metrics(
                y_true = scores_df['Validated_RAI_Score'],
                y_pred = scores_df['llm_RAI_score']
            )
            print(f"Binary metrics for {model_name}:")
            print(round_metrics(binary_metrics_complete))
            print(f"Continuous metrics for {model_name}:")
            print(round_metrics(continuous_metrics_complete))
        
        else:
            print(f"Missing cases found for {model_name}. Proceeding with imputation.")
            mask_missing = scores_df["llm_RAI_score"].isna()
            imputed_df_all = mult_impute_with_randint(scores_df, mask_missing, min_val, max_val, n_sims)
            spearman_samps, rmse_samps, accuracy_samps, precision_samps, recall_samps, f1_samps, pr_auc_samps = get_imputed_metrics_samps(imputed_df_all)
            print(f"Imputation metrics for {model_name}:")
            print(f"Spearman: {np.mean(spearman_samps):.4f}, "
                f"RMSE: {np.mean(rmse_samps):.4f}")
            print(f"Acc: {np.mean(accuracy_samps):.4f}, "
                f"Prec: {np.mean(precision_samps):.4f}, "
                f"Recall: {np.mean(recall_samps):.4f}, "
                f"F1 Score: {np.mean(f1_samps):.4f}, "
                f"PR AUC: {np.mean(pr_auc_samps):.4f}")
        
        print("\n" + "="*50 + "\n")

In [ ]:
# 0 to 81
print_metrics_for_all_models(all_df, all_model_names, min_RAI, max_RAI, n_sims)

In [ ]:
# 0 to 47
print_metrics_for_all_models(all_df, all_model_names, min_validated_RAI, max_validated_RAI, n_sims)